In [1]:
"""
Run this block to import the relevant modules. 
"""

import csv
from Vienna_Wrapper_DSY import *

In [2]:
"""
This block contains the functions used to generate the predictions. 

Run this block before running the subsequent blocks.
"""

def free_5_prime_bases(struct):
    bases = 0
    index = 0
    current = struct[index]
    
    while  current == '.':
        index += 1
        bases += 1
        current = struct[index]
    return bases
        
    
    
def gRNA_eval(sequence, space_length=20, upstream=0, truncate=False, kinetic=True, robust=False, scRNA=True, tracr = False, aptseq=False):
    sequence = sequence.upper()
    sequence = sequence.replace('T', 'U')
    upstruct = (upstream * '.')
    
    if aptseq:
        aptseq = aptseq.upper()
        aptseq = aptseq.replace('T', 'U')
        aptindex = sequence.find(aptseq)
        aptstruct = RNAfold(aptseq)[1].replace('.','x')
        upstruct = ('.' * aptindex) + aptstruct + ((upstream - (len(aptseq) + aptindex)) * '.')
    
    if truncate:
        sequence = sequence[:upstream + space_length + truncate]
    HANDLE = '(((((((.((((....))))...)))))))'#If you're using a different handle than normal, this is where you put it in(((((((.(((((((((((.....)))))))))))...)))))))'#'(((((((.(((((((((....)))))))))...)))))))'#
    if scRNA:
        MS2 = '(((((((.((....)))))))))'#Ditto for the MS2 (or other activator-binding) RNA'(((((((((((.....)))))))))))(((((((...)))))))'#'(((((((((((.((((......)))))))))))(((((((((((((.((((......))))))))))))))))).))))'#'...((((......))))......'#
    else:
        MS2 = '((((((...))))))'
    fstruct = upstruct + ((len(sequence) - upstream) * '.')
    MFE, MFS = RNAfold(sequence, struct=fstruct)
    fstruct = upstruct + ((len(sequence) - upstream) * '.')
    DUPE = RNAeval_dup(sequence[upstream:space_length+upstream], rev_comp(sequence[upstream:space_length+upstream]), (space_length * '(') , (space_length * ')'))
    SPACE, SPACS = RNAfold(sequence, struct=(upstream * '.') + (space_length * 'x') + ((len(sequence) - space_length - upstream) * '.'))
    HANDE, HANDS = RNAfold(sequence, struct=(upstream * '.') + (space_length * '.') + HANDLE + ((len(sequence) - len(HANDLE) - space_length - upstream) * '.'))
    
    SHE, SHS = RNAfold(sequence, struct=(upstream * '.') + (space_length * 'x') + HANDLE + (18 * 'x') + ((len(sequence) - len(HANDLE) - space_length - upstream - 18) * '.'))
    
    BESTE, BESTS = RNAfold(sequence, struct=(upstream * '.') + (space_length * '.') + HANDLE + (18 * 'x') + ((len(sequence) - len(HANDLE) - space_length - upstream - 18) * '.'))
    BEST_BARR = findpath(sequence, BESTS, SHS)
    BEST_DDG = BESTE - SHE
    
    X = pf_stab(sequence, struct=fstruct)[0]
    x2 = pf_stab(sequence)[0]
    Y = pf_stab(sequence, struct=(upstruct + (space_length * '.') + HANDLE + ((len(sequence) - len(HANDLE) - space_length - upstream) * '.')))[0]
    handle_frac = pf_frac(Y, X)
    
    if MS2 in MFS:
        ms2 = True
    else:
        ms2 = False
    handle_e = HANDE - MFE
    net_e = DUPE + SPACE - MFE
    space_net_e = DUPE + SHE - HANDE
    space_frac = BP_content(MFS[upstream:space_length+upstream])
    seed_space_avail = (1 - BP_content(HANDS[upstream + (space_length - min(8, space_length)):space_length+upstream])) * min(8, space_length)
    spacer_e = SPACE - MFE
    handle_spacer_e = SHE - HANDE
    new_net_e = DUPE + SHE - MFE
    thermo5 = free_5_prime_bases(MFS)
    hand_barr = findpath(sequence, MFS, HANDS)
    space_barr = findpath(sequence, HANDS, SHS)
    bind_barr = findpath(sequence, MFS, SHS)

#     if truncate==91:
#         print MFEpath(sequence, mfthreshold=7.8, verbose=True)

    if kinetic:
        if robust:
            path = MFEpath(sequence, mfthreshold=7.8, verbose=True, apt_thresh=0.00000001) ####changed from 7.8
        else:
            path = MFEpath(sequence, mfthreshold=7.8, verbose=True)
            if not path[0]:
                return None
        #print path
        kin_hand_barr =  findpath(sequence, path[0][5][-2] + '.', HANDS)
        kin_bind_barr = findpath(sequence, path[0][5][-2] + '.', SHS)
        if HANDLE in path[0][5][-2] and HANDLE in path[0][5][-1]:
            kin_hand = True
        else:
            kin_hand = False

        if MS2 in path[0][5][-2] and MS2 in path[0][5][-1]:
            kin_ms2 = True
        else:
            kin_ms2 = False
        kin5 = free_5_prime_bases(path[0][5][-2])
        convergence = path[0][4]
        kin_bind_ddg = SHE - RNAeval(sequence, path[0][5][-2] + '.')
        if robust:
            five_frac = local_barriers(sequence, ref=SHS, gap=1.4, threshold=5.)
            ten_frac = local_barriers(sequence, ref=SHS, gap=1.4, threshold=10.)
            kin_frac = np.prod(path[0][-4])
        else:
            kin_frac = 1000
            five_frac = 1000
            ten_frac = 1000
    else:
        kin_hand = 1000
        kin_ms2 = 1000
        kin_hand_barr = 1000
        kin5 = 1000
        kin_bind_barr = 1000
        five_frac = 1000
        kin_bind_ddg = 1000
        ten_frac = 1000
        convergence = 1000
        kin_frac = 1000
        five_frac = 1000
        
    
    GC = GC_content(sequence[:20])
    seed_GC = GC_content(sequence[12:20])
    spacer_ddg = SHE - HANDE
    bind_ddg = SHE - MFE
    
    if tracr:
        seq1 = sequence
        s1 = MFS
        seq2 = 'GGAACCAUUCAAAACAGCAUAGCAAGUUAAAAUAAGGCUAGUCCGUUAUCAACUUGAAAAAGUGGCACCGAGUCGGUGCUUUUUUU'
        s2 = '......................((((((...((((.(......).)))).)))))).(((((.(((((((...))))))).)))))'
        dup = RNAcofold(seq1, seq2)
        E1 =  MFE
        E2 = -17.4
        E_net = dup[0] - (E1 + E2)

        barrier_estimate = findpath(seq1 + 'AAAA' + seq2, s1 + '....' + s2, dup[1] + '....' + dup[2])
        if MS2 in dup[1]:
            dup_ms2 = True
        else:
            dup_ms2 = False
        if HANDLE in dup[1]:
            dup_handle = True
        else:
            dup_handle = False
    else:
        E_net, barrier_estimate, dup_handle, dup_ms2 = 1000, 1000, 1000, 1000
    
    
    return round(net_e,2), round(handle_e, 2), round(space_frac,2), round(space_net_e,2), handle_frac, round(spacer_e,2), round(handle_spacer_e,2), round(DUPE,2), round(new_net_e,2), ms2, seed_space_avail, kin_hand, kin_ms2, round(kin_hand_barr,2), kin5, thermo5, round(hand_barr,2), round(space_barr,2), round(bind_barr,2), round(kin_bind_barr,2), round(five_frac,3), round(ten_frac,3), round(convergence,3), round(kin_frac,3), round(GC,3), round(seed_GC,3),round(spacer_ddg,3), round(bind_ddg,3), round(kin_bind_ddg,3), round(E_net, 2), barrier_estimate, dup_handle, dup_ms2, BEST_BARR, BEST_DDG

In [3]:
"""
Update 'CSV' to point to the current location of the correctly-formatted input file.
Use the example file 'Promoters_paper_example.csv' as a template.

The output file will be written to the same location with "_analysis" appended.
See Column B (Folding Barrier) in output for the key parameter in the 2024 paper.
"""

CSV = 'gRNAs_2_screen/Promoters_paper_example.csv'

SCRNA = True #Change to False if analyzing sgRNAs instead

KINETIC = False

######################################################################################################################
output = []
with open(CSV, 'rU') as data:
    read = csv.reader(data)
    read.next()
    for line in read:
        print line[0]
        a, b, c, d, e, f, g, h, i, j, k, l, m, n, o, p, q, r, s, t, u, v, w, x, y, z, aa, bb, cc, dd, ee, ff, gg, hh, ii = gRNA_eval(line[8], int(line[1]), int(line[9]), kinetic=KINETIC, scRNA=SCRNA, tracr=False)
        output.append([line[0], s, q, r, i, bb, h, e, a, b, c, d, f, g, j, k, l, m, n, o, p, t, u, v, w, x, y, z, aa, cc, dd, ee, ff, gg, hh, ii, line[6]])
        

Labels = ['Name', 'Folding Barrier', 'Handle Barrier', 'Spacer Barrier', 'Net Binding Energy', 'Folding Energy', 'Binding Energy', 'Handle Fraction', 'Net Energy', 'Handle Penalty', 'Spacer Structure Fraction', 'Spacer Net Energy', 
         'Spacer Penalty', 'Handle-Spacer Penalty', 
          'MS2 in MFE?', 'Available Seed Bases', 'Kinetic Handle?', 'Kinetic MS2?', 'Kinetic Handle Barrier', 'Thermo 5 prime bases', 'Kinetic 5 prime bases', 'Kinetic Bind Barrier', 'Local Fraction (5)', 'Local Fraction (10)', 'MFEpath Convergence', 'Stringent Convergence', 'Spacer GC Content', 'Seed GC Content', 'Spacer ddG', 'Kinetic Bind ddG', 'tracer duplex net energy', 'tracer duplex (ROUGH) barrier', 'tracr duplex handle formed?', 'tracr duplex ms2 formed?', 'Spacer-only Barrier','Spacer-only ddG', 'Fluorescence']

outfile = CSV.split('.')[0] + '_analysis.csv'
with open(outfile, 'wb') as OUTFILE:
    C = csv.writer(OUTFILE)
    C.writerow(Labels)
    for out in output:
        C.writerow(out)
print(outfile)

Tet35-10.1
Tet35-10.2
Tet35-10.3
Tet35-10.4
Tet35-10.5
Tet35-10.6
Tet35-10.7
Tet35-10.8
Tet35-10.9
Tet35-10.10
Tet35-10.11
Tet35-10.12
Tet35-10.13
Tet35-10.14
Tet35-10.15
Tet35-10.16
Tet35-12.1
Tet35-12.2
Tet35-12.3
Tet35-12.4
Tet35-12.5
Tet35-12.6
Tet35-12.7
Tet35-12.8
Tet35-12.9
Tet35-12.10
Tet35-12.11
Tet35-12.12
Tet35-12.13
Tet35-12.14
Tet35-12.15
Tet35-12.16
Tet35-14.1
Tet35-14.2
Tet35-14.3
Tet35-14.4
Tet35-14.5
Tet35-14.6
Tet35-14.7
Tet35-14.8
Tet35-14.9
Tet35-14.10
Tet35-14.11
Tet35-14.12
Tet35-14.13
Tet35-14.14
Tet35-14.15
Tet35-14.16
Tet35-16.1
Tet35-16.2
Tet35-16.3
Tet35-16.4
Tet35-16.5
Tet35-16.6
Tet35-16.7
Tet35-16.8
Tet35-16.9
Tet35-16.10
Tet35-16.11
Tet35-16.12
Tet35-16.13
Tet35-16.14
Tet35-16.15
Tet35-16.16
gRNAs_2_screen/Tet-NFB-chimera-scRNAs_analysis.csv
